In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 

In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
#I don´t understand why header don´t works 
#header works with argument 0 
dataframe = pandas.read_csv("sonar.csv", header=0) 
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [4]:
#print(X,Y)

In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [6]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer= "normal" , activation= "relu" ))
    model.add(Dense(1, kernel_initializer= "normal" , activation= "sigmoid" ))
    # Compile model
    model.compile(loss= "binary_crossentropy" , optimizer= "adam" , metrics=[ "accuracy" ])
    return model

def create_baseline2():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=60, kernel_initializer= "normal" , activation= "relu" ))
    model.add(Dense(1, kernel_initializer= "normal" , activation= "sigmoid" ))
    # Compile model
    model.compile(loss= "binary_crossentropy" , optimizer= "adam" , metrics=[ "accuracy" ])
    return model

def create_baseline3():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer= "normal" , activation= "relu" ))
    model.add(Dense(30, kernel_initializer= "normal" , activation= "relu" ))
    model.add(Dense(1, kernel_initializer= "normal" , activation= "sigmoid" ))
    # Compile model
    model.compile(loss= "binary_crossentropy" , optimizer= "adam" , metrics=[ "accuracy" ])
    return model

In [7]:
# evaluate model with standardized dataset
#estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

el resultado varia al 78%, no se si sea conveniente hacer un for para sacar un estimativo de la vianza de los resultados y los errores 

podemos arreglar el resultado anterior standarizando la data 


In [8]:
estimators = []
estimators.append(( "standardize" , StandardScaler()))
estimators.append(( "mlp" , KerasClassifier(build_fn=create_baseline, 
                                            epochs=100,batch_size=5, verbose=0)))

estimators2 = []
estimators2.append(( "standardize" , StandardScaler()))
estimators2.append(( "mlp" , KerasClassifier(build_fn=create_baseline2, 
                                            epochs=100,batch_size=5, verbose=0)))
estimators3 = []
estimators3.append(( "standardize" , StandardScaler()))
estimators3.append(( "mlp" , KerasClassifier(build_fn=create_baseline3, 
                                            epochs=100,batch_size=5, verbose=0)))
 

In [9]:
estimators

[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('mlp',
  <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier at 0x19c0dc9f6c8>)]

In [10]:
pipeline = Pipeline(estimators)
pipeline2 = Pipeline(estimators2)
pipeline3 = Pipeline(estimators3)

In [11]:
#pipeline

In [12]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [13]:
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.04% (5.53%)


resultado a mejorar

In [14]:
results = cross_val_score(pipeline2, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.52% (4.92%)


Con menos neuronas en la entrada el proceso de entrenamiento deberia ser mas rapido(porque hay menos neuronas que entrenar) ademas deberia haber un ligero aumento de la precisión media estimada importante reducción de la desviación estándar (dispersión media) de las puntuaciones de precisión del modelo. 

In [23]:
results = cross_val_score(pipeline3, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 86.04% (4.00%)


Una topología de red neuronal con más capas ofrece más oportunidades para que la red extraiga características clave y las recombine de formas no lineales útiles.

La idea es que la red tenga la oportunidad de modelar todas las variables de entrada antes de que se produzca un cuello de botella y se vea obligada a reducir a la mitad la capacidad de representación